# Building a chatbot

In [ ]:
$ mkdir backend
$ cd backend
$ virtualenv env
$ source env/bin/activate
$ pip install django djangorestframework
$ django-admin.py startproject chatbot_api
$ cd chatbot_api
$ python manage.py startapp message

When it's done, we need to add Django Rest Framework to the installed apps parameter of our project, in chatbot_api/chatbot_api/settings.py :

In [ ]:
# chatbot_api/chatbot_api/settings.py

INSTALLED_APPS = (
    ...
    'rest_framework',
    'Message.apps.MessageConfig',
)

In [ ]:
$ python manage.py migrate
$ python manage.py runserver

Now we can create a model, it's the class of our object. Here we want to store messages including some text, the firstname of the user and the date of creation.

In [ ]:
# chatbot_api/message/models.py

from django.db import models

class Message(models.Model):
    created = models.DateTimeField(auto_now=True)
    text = models.TextField()
    firstname = models.CharField(max_length=128, default="anonymous")


In [ ]:
$ python manage.py migrate

When you receive data on the server, you want to make sure that its correct. Django let us create a serializer, it will read the data received from a request, and translate it as a model. It will also be able to check if the data is valid and serialize samples coming from the dataset to be sent back to the frontend.

In [ ]:
# chatbot_api/message/serializers.py

from rest_framework import serializers
from message.models import Message

class MessageSerializer(serializers.ModelSerializer):
    class Meta:
        model = Message 
        fields = ('text', 'firstname')


Good news, we can now create an API view. It's the API interface of a model.

Ours will be fairly simple, we only want to be able to receive messages, one by one, from the front-end.

In [ ]:
# chatbot_api/message/views.py

from message.models import Message
from message.serializers import MessageSerializer
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status

class MessageView(APIView):
    def post(self, request, format=None):
        serializer = MessageSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()             
            return Response({"speech_answer": "J'ai bien reçu ton message."}, status=status.HTTP_200_OK)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


Once we have our view, we want to actually be able to interact with that view. So we have to register it, to add it to our list of urls.

In [ ]:
# chatbot_api/chatbot_api/urls.py

...
from message.views import MessageView

urlpatterns = [
    ...
    url(r'^message/$', MessageView.as_view()),
]


We're almost done. If you try to run the server, you'll see that it doesn't respond anything intersting. There's no chatbot right now.

Let's start an agent on DialogFlow: https://console.dialogflow.com/

Next, we need to build a method that is going to call Dialogflow whenever you receive something. Use your own client key in that method.

In [ ]:
# chatbot_api/message/helpers/dialogflow.py

import requests

def query_dialogflow(message):
    client_key = "<CLIENT_KEY>"
    headers = {"Authorization": "Bearer " + client_key}  
    payload = {"lang": "fr", "query": message, "v":20150910, "sessionId":1}
    r = requests.get('https://api.dialogflow.com/v1/query', params=payload, headers=headers)
    return r


Let's update our view to use that method.

In [ ]:
# chatbot_api/message/views.py

from message.models import Message
from message.serializers import MessageSerializer
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status
from message.helpers.dialogflow import query_dialogflow

class MessageView(APIView):
    def post(self, request, format=None):
        serializer = MessageSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            r = query_dialogflow(serializer.data.get('text'))            
            json_response = r.json()
            response = json_response.get('result', {}).get("fulfillment", {}).get("speech", "Je n'ai pas compris...")              
            return Response({"speech_answer": response}, status=status.HTTP_200_OK)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


Et voilà !

Let's run the server one last time.

In [ ]:
$ python manage.py runserver